# 데이터 삽입

In [44]:
from pymongo import MongoClient #MongoDB - Python 연결
from faker import Faker #더미 데이터 생성
from datetime import datetime

In [45]:
faker = Faker()

In [46]:
client = MongoClient('mongodb://localhost:27017/')
db = client['testBookList'] # 'testBookList' 데이터베이스 사용

# 함수 작성

In [124]:
def drop_all_collections(db) :
    for name in db.list_collection_names() :
        db.drop_collection(name)
        
def generate_dummy_book(db) :
    return {'title' : faker.sentence(3),
            'author' : faker.word(ext_word_list=['Akira Kurosawa', 'Federico Fellini', 'Ingmar Bergman',\
                    'Jean-Luc Godard', 'Andrei Tarkovsky','Alejandro González Iñárritu','Pedro Almodóvar',\
                        'Wong Kar-wai', 'Park Chan-wook', 'Hayao Miyazaki']),
            'year' : faker.pyint(2000,2024),
            'genre' : faker.word(ext_word_list=['Action', 'Science Fiction', 'Comedy', 'Melodrama', 'Drama', 'Thriller', 'Horror', 'Animation', 'Documentary', 'Fantasy', 'Musical'])
            }
    
def generate_dummy_movie(db) :
    return {'title' : faker.sentence(3),
            'director' : faker.word(ext_word_list=['Akira Kurosawa', 'Federico Fellini', 'Ingmar Bergman',\
                    'Jean-Luc Godard', 'Andrei Tarkovsky','Alejandro González Iñárritu','Pedro Almodóvar',\
                        'Wong Kar-wai', 'Park Chan-wook', 'Hayao Miyazaki']),
            'year' : faker.pyint(2000,2024),
            'rating' : faker.pyfloat(left_digits=1, right_digits=1,positive=True),
            'genre' : faker.word(ext_word_list=['Action', 'Science Fiction', 'Comedy', 'Melodrama', 'Drama', 'Thriller', 'Horror', 'Animation', 'Documentary', 'Fantasy', 'Musical'])
            }
    
def getnerate_dummy_user(db) :
    return {'user_id' : faker.pyint(0,100),
            'action' : faker.word(ext_word_list=['click', 'view','purchase']),
            'timestamp' : faker.date_time_between(start_date=datetime(2000,1,1), end_date=datetime.now()),
            }

## CRUD

In [133]:
def insert_dummy_data(db, num) :
    db['books'].insert_many([generate_dummy_book(db) for _ in range(num)])
    db['movies'].insert_many([generate_dummy_movie(db) for _ in range(num)])
    db['user_actions'].insert_many([getnerate_dummy_user(db) for _ in range(num)])
    
def select_books_genre(db,genre) :
    book_collection = db.books
    qurey = {'genre' : genre}
    projection = {"_id" : 0, 'title' : 1, 'author' : 1, 'genre' : 1}
    books = book_collection.find(qurey, projection)
    for book in books :
        print(book)
        
def get_director_avg_rating(db) :
    movies_collection = db.movies
    pipeline = [
        {"$group": {"_id": "$director", "average_rating": {"$avg": "$rating"}}},
        {"$sort": {"average_rating": -1}}
    ]
    results = movies_collection.aggregate(pipeline)
    for result in results:
        print(result)
        
def get_user_action_by_id(db, user_id, limit=5) :
    user_actions_collection = db.user_actions
    query = {"user_id": user_id}
    sort_criteria = [("timestamp", -1)]

    actions = user_actions_collection.find(query).sort(sort_criteria).limit(limit)
    for action in actions:
        print(action)
        
def get_numOfBooks_byYear(db):
    books_collection = db.books
    pipeline = [
        {"$group": {"_id": "$year", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    results = books_collection.aggregate(pipeline)
    for result in results:
        print(result)
    
def update_user_action(db, user_id,old_action, new_action, date) :
    user_actions_collection = db.user_actions
    query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}
    result = user_actions_collection.update_many(query, update)
    print(f"Updated {result.modified_count} documents.")

In [126]:
drop_all_collections(db)
insert_dummy_data(db, 1000)

## 문제1

- **문제 설명**:
사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
"fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [127]:
select_books_genre(db, 'Fantasy')

{'title': 'Dog half positive simple.', 'author': 'Hayao Miyazaki', 'genre': 'Fantasy'}
{'title': 'Man sign change.', 'author': 'Alejandro González Iñárritu', 'genre': 'Fantasy'}
{'title': 'Option work only.', 'author': 'Alejandro González Iñárritu', 'genre': 'Fantasy'}
{'title': 'Rock less.', 'author': 'Federico Fellini', 'genre': 'Fantasy'}
{'title': 'Together yourself a.', 'author': 'Alejandro González Iñárritu', 'genre': 'Fantasy'}
{'title': 'Pull collection their.', 'author': 'Federico Fellini', 'genre': 'Fantasy'}
{'title': 'Easy present.', 'author': 'Jean-Luc Godard', 'genre': 'Fantasy'}
{'title': 'Hand.', 'author': 'Park Chan-wook', 'genre': 'Fantasy'}
{'title': 'Raise pick.', 'author': 'Andrei Tarkovsky', 'genre': 'Fantasy'}
{'title': 'South behind.', 'author': 'Alejandro González Iñárritu', 'genre': 'Fantasy'}
{'title': 'Special character.', 'author': 'Alejandro González Iñárritu', 'genre': 'Fantasy'}
{'title': 'Understand summer.', 'author': 'Ingmar Bergman', 'genre': 'Fantas

## 문제2

- **문제 설명**:
각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**:
모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [128]:
get_director_avg_rating(db)

{'_id': 'Jean-Luc Godard', 'average_rating': 5.285869565217391}
{'_id': 'Pedro Almodóvar', 'average_rating': 5.268817204301075}
{'_id': 'Andrei Tarkovsky', 'average_rating': 5.206593406593407}
{'_id': 'Wong Kar-wai', 'average_rating': 5.126724137931035}
{'_id': 'Hayao Miyazaki', 'average_rating': 5.115044247787611}
{'_id': 'Federico Fellini', 'average_rating': 5.084313725490197}
{'_id': 'Ingmar Bergman', 'average_rating': 5.0523364485981315}
{'_id': 'Alejandro González Iñárritu', 'average_rating': 4.783333333333333}
{'_id': 'Akira Kurosawa', 'average_rating': 4.529126213592233}
{'_id': 'Park Chan-wook', 'average_rating': 4.4037037037037035}


## 문제3

- **문제 설명**:
특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [129]:
get_user_action_by_id(db, 13)

{'_id': ObjectId('66324416a724b684af516338'), 'user_id': 13, 'action': 'view', 'timestamp': datetime.datetime(2020, 7, 17, 3, 26, 51, 801000)}
{'_id': ObjectId('66324416a724b684af5162e8'), 'user_id': 13, 'action': 'view', 'timestamp': datetime.datetime(2019, 6, 3, 12, 28, 46, 721000)}
{'_id': ObjectId('66324416a724b684af51659a'), 'user_id': 13, 'action': 'click', 'timestamp': datetime.datetime(2012, 12, 15, 20, 5, 23, 231000)}
{'_id': ObjectId('66324416a724b684af5165ab'), 'user_id': 13, 'action': 'view', 'timestamp': datetime.datetime(2008, 8, 12, 8, 16, 38, 368000)}
{'_id': ObjectId('66324416a724b684af51634a'), 'user_id': 13, 'action': 'click', 'timestamp': datetime.datetime(2008, 5, 17, 11, 29, 38, 93000)}


## 문제4

- **문제 설명** :
데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [130]:
get_numOfBooks_byYear(db)

{'_id': 2008, 'count': 56}
{'_id': 2021, 'count': 50}
{'_id': 2016, 'count': 47}
{'_id': 2001, 'count': 46}
{'_id': 2020, 'count': 46}
{'_id': 2022, 'count': 43}
{'_id': 2005, 'count': 43}
{'_id': 2009, 'count': 42}
{'_id': 2007, 'count': 42}
{'_id': 2019, 'count': 41}
{'_id': 2014, 'count': 41}
{'_id': 2013, 'count': 41}
{'_id': 2024, 'count': 39}
{'_id': 2000, 'count': 39}
{'_id': 2017, 'count': 39}
{'_id': 2018, 'count': 38}
{'_id': 2011, 'count': 37}
{'_id': 2010, 'count': 37}
{'_id': 2023, 'count': 36}
{'_id': 2002, 'count': 35}
{'_id': 2003, 'count': 34}
{'_id': 2015, 'count': 34}
{'_id': 2012, 'count': 33}
{'_id': 2004, 'count': 32}
{'_id': 2006, 'count': 29}


## 문제5

- **문제 설명**:
특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [136]:
update_user_action(db, 13,'view', 'seen', datetime(2023,4,10))

Updated 5 documents.
